In [1]:
import os 

In [2]:
%pwd

'/Users/sanjaymahto/NLP-1/Text Summarizer/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/sanjaymahto/NLP-1/Text Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [6]:
from src.textSummarizer.constants import * 
from src.textSummarizer.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir, 
            data_path=config.data_path, 
            model_path=config.model_path, 
            tokenizer_path=config.tokenizer_path, 
            metric_file_name=config.metric_file_name
        )
        return model_evaluation_config
        

In [20]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import  load_from_disk
import torch
import pandas as pd 
from tqdm import tqdm
import evaluate

In [23]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """Split the datset into smaller batches that we can process simultaneously 
        Yield successive bach sized chunks from list_of_elements"""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i+batch_size]
            
    def calculate_metric_on_tes_ds(self, dataset, metric, model, tokenizer, 
                                   batch_size=16, device = torch.device("mps" if torch.backends.mps.is_available() and torch.backends.mps.is_built() else "cuda" if torch.cuda.is_available() else "cpu"), 
                                   column_text = 'article', 
                                   column_summary='highlights'):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))


        for article_batch, target in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
            
            inputs = tokenizer(article_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")

            summaries = model.generate(input_ids=inputs['input_ids'].to(device),
                                attention_mask=input['attention_mask'].to(device),
                                length_penalty=0.8,
                                num_beams=8,
                                max_length=128)

            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summaries]
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

            metric.add_batch(predictions=decoded_summaries, references=target)
        
        # Finally compute and return the ROUGE Scores, 
        score = metric.compute()
        return score
    
    def evaluate(self):
        device = torch.device("mps" if torch.backends.mps.is_available() and torch.backends.mps.is_built() else "cuda" if torch.cuda.is_available() else "cpu")
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
        
        # loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
        rouge_metric = evaluate.load('rouge')
        
        score = self.calculate_metric_on_tes_ds(
            dataset_samsum_pt['test'][0:10], rouge_metric, model_pegasus, tokenizer, batch_size=2, device=device,
            column_text="dialogue",
            column_summary="summary"  
            
        )
        rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
        
        df = pd.DataFrame(rouge_dict, index=['pegasus'])
        df.to_csv(self.config.metric_file_name, index=False)

In [32]:
import evaluate                    # ← new import
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch, pandas as pd
from tqdm import tqdm

# … ModelEvaluationConfig dataclass assumed above …

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """Split the datset into smaller batches that we can process simultaneously 
        Yield successive bach sized chunks from list_of_elements"""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i+batch_size]
            
    def calculate_metric_on_tes_ds(self, dataset, metric, model, tokenizer, 
                                   batch_size=16, device = torch.device("mps" if torch.backends.mps.is_available() and torch.backends.mps.is_built() else "cuda" if torch.cuda.is_available() else "cpu"), 
                                   column_text = 'article', 
                                   column_summary='highlights'):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))


        for article_batch, target in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
            
            inputs = tokenizer(article_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")

            summaries = model.generate(input_ids=inputs['input_ids'].to(device),
                                attention_mask=inputs['attention_mask'].to(device),
                                length_penalty=0.8,
                                num_beams=8,
                                max_length=128)

            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summaries]
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

            metric.add_batch(predictions=decoded_summaries, references=target)
        
        # Finally compute and return the ROUGE Scores, 
        score = metric.compute()
        return score

    def evaluate(self):
        device = torch.device(
            "mps" if torch.backends.mps.is_available() and torch.backends.mps.is_built()
            else "cuda" if torch.cuda.is_available()
            else "cpu"
        )
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        dataset = load_from_disk(self.config.data_path)

        rouge_metric = evaluate.load("rouge")           # ← already the new call

        score = self.calculate_metric_on_tes_ds(       # (rename optional)
            dataset["test"][:10],
            rouge_metric,
            model,
            tokenizer,
            batch_size=2,
            device=device,
            column_text="dialogue",
            column_summary="summary",
        )

        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        rouge_dict = {rn: score[rn].mid.fmeasure for rn in rouge_names}

        pd.DataFrame(rouge_dict, index=["pegasus"]).to_csv(
            self.config.metric_file_name, index=False
        )


In [33]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e 

[2025-05-02 15:26:33,460 - INFO - common - yaml file: config/config.yaml loaded successfully]
[2025-05-02 15:26:33,470 - INFO - common - yaml file: params.yaml loaded successfully]
[2025-05-02 15:26:33,471 - INFO - common - Directory created: artifacts]
[2025-05-02 15:26:33,476 - INFO - common - Directory created: artofacts/model_evaluation]


  0%|          | 0/5 [51:10:51<?, ?it/s]


KeyboardInterrupt: 